In [0]:
from pyspark.sql.functions import count, max,avg,sum

df_maintenance = spark.table("silver_maintenance_logs")

df_fact_maintenance = (
    df_maintenance
    .groupBy("maintenance_asset_id")
    .agg(
        count("maintenance_date").alias("maintenance_count"),
        max("maintenance_date").alias("last_maintenance_date"),
        sum("CostUSD").alias("total_maintenance_cost"),
        avg("CostUSD").alias("avg_maintenance_cost")
    )
    )


df_fact_maintenance.write \
    .mode("overwrite") \
    .format("delta") \
    .save("abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/Gold/gold_fact_maintenance_activity_delta")

spark.sql("""
    CREATE TABLE IF NOT EXISTS gold_fact_maintenance_activity
    USING DELTA
    LOCATION 'abfss://sedpcontainer@sedpstorageaccount.dfs.core.windows.net/Gold/gold_fact_maintenance_activity_delta'
""")

DataFrame[]